# Transport File

Настоящая обработка создана как аналог файла "Transport 22 XX with query", и, при необходимости, результаты могут быть сверены и должны быть идентичны друг другу
*(прим. автора)*

In [1]:
# Импорт модулей
import pandas as pd
import numpy as np

from datetime import datetime, date, time
import openpyxl
from openpyxl import load_workbook

import xlsxwriter

# отключим предупреждения
import warnings
warnings.simplefilter('ignore')

#суммы будут отражаться до второго знака
pd.options.display.float_format = '{:,.2f}'.format 

### Импорт данных

Загрузим в память необходимые дата-сеты

Конечно, перед началом обработки нижеуказанные файлы должны быть обновлены

**!Перед началом работы обязательно проверить пути/месяцы и, при необходимости, заменить на нужные!**

In [2]:
#Загрузка плоского, берём лист PL
get_pl = pd.read_excel(r"S:\Actual\Controlling\Плоский для P&L_2022.xlsm", sheet_name = 'P&L')

In [3]:
#забираем лист Manually из Транспортного файла
manually = pd.read_excel(r"S:\Actual\_Reporting 2022\22 12\Trs\Transport 22 12 with queryOPT.xlsx", sheet_name = 'Manually')

In [4]:
uat_rev_exp = pd.read_excel(r"S:\Actual\_Reporting 2022\22 12\Trs\UAT 22 12_V_LTL.xlsx", sheet_name = 'UAT_Rev&Exp')

In [5]:
bonus_variances = pd.read_excel(r"S:\Actual\_Reporting 2022\22 12\Trs\UAT_vs_UPP recon 2212.xlsx", sheet_name = 'BonusVariance')

In [6]:
uat_statistics = pd.read_excel(r"S:\Actual\_Reporting 2022\22 12\Trs\UAT 22 12_V_LTL.xlsx", sheet_name = 'UAT_Statistics')

In [7]:
writer = pd.ExcelWriter(r'S:\Actual\_Reporting 2022\22 12\Trs\Transport 22_12 PY.xlsx', engine='xlsxwriter')

На данном этапе импорт данных завершен

### Подготовка, формирование и расчет необходимых данных

In [8]:
#убираем ненужные дубликаты
uat_statistics = uat_statistics.drop_duplicates(keep=False)

На данном этапе необходимо поменять цифру месяца на текущий/нужный

In [9]:
month = 12  #указываем номер месяца, который необходим

**UPP Expenses**

In [10]:
upp_expenses =  get_pl[((get_pl['Month'] == month) & (get_pl['Комментарий'] != 'Справка ОФК от 04.04.22 | Рекласс затрат | RE  02 рекласс T-T0 2022 03.msg | 02 рекласс T-T0 2022 03.xlsx') & (get_pl['BussinessType'] == 'Trs') & (get_pl['Код бизнеса'] != 'E'))]

In [11]:
def custom1(upp_expenses):
    if (upp_expenses['Номенклатурная группа'] == 'Данон') and (upp_expenses['Код бизнеса'] == 'T'):
        return 'DNN T1'
    elif (upp_expenses['Номенклатурная группа'] == 'Данон') and (upp_expenses['Код бизнеса'] == 'T0'):
        return 'DNN Other'
    elif (upp_expenses['Номенклатурная группа'] == 'Центральный офис') and (upp_expenses['Код бизнеса'] == 'T'):
        return 'DNN T1'
    elif (upp_expenses['Номенклатурная группа'] == 'Данон') and (upp_expenses['Код бизнеса'] == 'M'):
        return 'DNN Milk'
    elif (upp_expenses['Контрагент'] == 'Корректировка'):
        return 'DNN T1'
    else:
        return upp_expenses['Контрагент']
        


In [12]:
upp_expenses['Client'] = upp_expenses.apply(custom1, axis=1)

In [13]:
upp_expenses = upp_expenses.groupby(['Month', 'AccType', 'Costs group', 'Номенклатурная группа', 'Client', 'Код региона', 'Код бизнеса', 'Calc items'])\
                  .agg({'NetAmount':'sum'})\
                  .reset_index()

upp_expenses.rename(columns = {'Код региона' : 'Region', 'Код бизнеса' : 'Code'}, inplace = True) 


In [14]:
upp_expenses['Region.Code'] = upp_expenses['Region'].astype(str) + "_" + upp_expenses['Code'].astype(str)
upp_expenses = upp_expenses[upp_expenses['NetAmount'] != 0]

**Manual**

In [15]:
manual = manually[["Source", 'Group', 'AccType', 'Принадлежность ТС', 'Region', 'Client', 'Milk', 'DNN/MC', 'Region.Code', 'NetAmount', 'Multipl']]

In [16]:
upp_subco = manual[manual['Group'] == 'Subcontractors cost']
upp_subco =upp_subco[['Group', 'Принадлежность ТС', 'Region', 'Client', 'Milk', 'DNN/MC', 'Region.Code', 'NetAmount']]
upp_subco.rename(columns = {'Group' : 'Costs group', 'NetAmount' : 'Net Amount'}, inplace = True) 

#### Расчет Bonus Variances & subcontractors_cost

In [17]:
bonus_variances = bonus_variances.groupby(['Customer', 'Region'])\
                  .agg({'Bonus':'sum'})\
                  .reset_index()

In [18]:
bonus_variances = bonus_variances.drop_duplicates() 
bonus_variances['Принадлежность ТС'] = 'Наемное'
bonus_variances.rename(columns = {'Customer' : 'Client', 'Region' : 'Region.Code'}, inplace = True) 

In [19]:
bonus_variances['Region']=bonus_variances['Region.Code'].str.split('_').str.get(0)

bonus_variances['Milk']=bonus_variances['Region.Code'].str.split('_').str.get(1)

In [20]:
subcontractor_cost = uat_rev_exp.drop('Revenue', 1)
subcontractor_cost = subcontractor_cost[subcontractor_cost['Subcontractor Cost'] != 0]


subco_cost = subcontractor_cost.merge(bonus_variances, left_on=['Client','Region.Code','Принадлежность ТС'],right_on = ['Client','Region.Code','Принадлежность ТС'], how='outer')

In [21]:
subco_cost['Bonus'].fillna(0, inplace=True) #заменим пропущенные значения  на 0

In [22]:
subco_cost['Net Amount'] = subco_cost['Subcontractor Cost'] + subco_cost['Bonus']
subco_cost['Costs group'] = 'Subcontractors cost'

In [23]:
subcontractors_cost = pd.concat([subco_cost,upp_subco], sort= False, axis=0)

#### Total TRS Revenue

In [24]:
uat_revenue = uat_rev_exp

In [25]:
uat_revenue = uat_revenue.drop(columns = ['Subcontractor Cost'],axis = 1)

In [26]:
uat_revenue = uat_revenue.drop(columns = ['Subcontractor Cost_'],axis = 1)

In [27]:
uat_revenue = uat_revenue[uat_revenue['Revenue'] != 0]
uat_revenue['Source'] = 'UAT'
uat_revenue = uat_revenue.drop_duplicates() 

In [28]:
upp_revenue = manual[manual['Group'] == 'Revenue']
upp_revenue = upp_revenue.drop(["Group", "AccType", "Multipl"], 1)
upp_revenue.rename(columns = {'NetAmount': 'Revenue'}, inplace = True) 

In [29]:
total_trs_revenue = pd.concat([uat_revenue, upp_revenue],axis=0)

In [30]:
total_trs_revenue.rename(columns = {'Revenue':'Net Amount'}, inplace = True) 

In [31]:
total_trs_revenue['Fin Amount'] = total_trs_revenue['Net Amount']
total_trs_revenue['Type'] = 'Rev'
total_trs_revenue['Costs group'] = 'Revenue'

In [32]:
total_trs_revenue = (total_trs_revenue.groupby(["Принадлежность ТС", "Region", "Client", "Milk", "DNN/MC", "Region.Code", "Type", "Costs group"])
                    .agg({'Net Amount':'sum', 'Fin Amount':'sum'})
                    .reset_index())

#### Формирование данных по Repair and Maintenance

In [33]:
repair_maintenance = upp_expenses[upp_expenses['Costs group'] == 'Repair & Maintenance'] 


In [34]:
repair_maintenance = repair_maintenance.drop('Client', 1)
repair_maintenance = repair_maintenance.drop('Region', 1)
repair_maintenance = repair_maintenance.drop('Code', 1)

In [35]:
repair_maintenance = repair_maintenance.merge(uat_statistics, left_on=['Region.Code'],right_on = ['Region.Code'], how='outer')

In [36]:
repair_maintenance = repair_maintenance[repair_maintenance['Принадлежность ТС'] == 'Собственное'] 

In [37]:
repair_maintenance['% of Total km'] = repair_maintenance['Total km'] / repair_maintenance['Sum of Total km']
repair_maintenance['Net Amount'] = repair_maintenance['NetAmount'] * repair_maintenance['% of Total km']

In [38]:
repair_maintenance = repair_maintenance[['Month', 'AccType', 'Costs group', 'Номенклатурная группа', 'Calc items', 'NetAmount', 'Region.Code', 'Принадлежность ТС', 'Region', 'Milk', 'Client', 'DNN/MC', 'Loaded km',  'Empty km',  'Total km', 'Turnover', 'Sum of Total km', 'Sum of Turnover', '% of Total km', 'Net Amount']]         

#### Salary Drivers

In [39]:
salary_drivers = upp_expenses[upp_expenses['Costs group'] == 'Salary Drivers'] 

In [40]:
salary_drivers = salary_drivers.drop(['Client','Region','Code'], 1)

In [41]:
salary_drivers = salary_drivers.merge(uat_statistics, left_on=['Region.Code'],right_on = ['Region.Code'], how='outer')

In [42]:
salary_drivers

,Month,AccType,Costs group,Номенклатурная группа,Calc items,NetAmount,Region.Code,Принадлежность ТС,Region,Milk,...,Total km,Turnover,Sum of Total km,Sum of Turnover,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,12.00,Exp,Salary Drivers,Данон,EBITDA,"1,978,265.62",BE_M,Собственное,BE,M,...,"184,221.00",353.15,"184,221.00",353.15,NaN,NaN,NaN,NaN,NaN,NaN
1,12.00,Exp,Salary Drivers,Данон,EBITDA,"1,520,238.67",YA_M,Собственное,YA,M,...,"114,391.00",272.63,"114,391.00",272.63,NaN,NaN,NaN,NaN,NaN,NaN
2,12.00,Exp,Salary Drivers,Данон,EBITDA,"20,550,992.11",BE_T,Собственное,BE,T,...,"14,295.00",16.32,"1,384,723.90","2,556.70",NaN,NaN,NaN,DNN T1,"2,924,409.50",NaN
3,12.00,Exp,Salary Drivers,Данон,EBITDA,"20,550,992.11",BE_T,Собственное,BE,T,...,"13,301.00",33.95,"1,384,723.90","2,556.70",NaN,NaN,NaN,DNN Other,57254,NaN
4,12.00,Exp,Salary Drivers,Данон,EBITDA,"20,550,992.11",BE_T,Собственное,BE,T,...,"24,871.00",56.70,"1,384,723.90","2,556.70",NaN,NaN,NaN,LTL,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,12.00,Exp,Salary Drivers,Данон,EBITDA,"7,724,107.06",YA_T,Арендованное,YA,T,...,"5,970.00",13.63,"156,248.00",306.94,NaN,NaN,NaN,NaN,NaN,NaN
114,12.00,Exp,Salary Drivers,Данон,EBITDA,"7,724,107.06",YA_T,Арендованное,YA,T,...,"9,689.00",28.80,"156,248.00",306.94,NaN,NaN,NaN,NaN,NaN,NaN
115,12.00,Exp,Salary Drivers,Данон,EBITDA,"7,724,107.06",YA_T,Арендованное,YA,T,...,"15,611.00",26.87,"156,248.00",306.94,NaN,NaN,NaN,NaN,NaN,NaN
116,12.00,Exp,Salary Drivers,Данон,EBITDA,"7,724,107.06",YA_T,Арендованное,YA,T,...,"1,618.00",3.57,"156,248.00",306.94,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
salary_drivers = salary_drivers.drop(['Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17'], 1)

In [44]:
salary_drivers= salary_drivers[salary_drivers['Принадлежность ТС'] == 'Собственное']

In [45]:
Salary_MARS_CCH = manual[manual['Group'] == 'Salary Drivers']

In [46]:
TechnicalRouteDNN = manually[['Client.1','Region.Code.1','Distance']]
TechnicalRouteDNN = TechnicalRouteDNN.head(2)

In [47]:
salary_drivers = salary_drivers.merge(Salary_MARS_CCH, left_on=['Client','Region.Code'],right_on = ['Client','Region.Code'], how='outer')

In [48]:
salary_drivers.rename(columns = {'NetAmount_y' : 'NetAmount.1', 'NetAmount_x' : 'NetAmount', 'Region_x' : 'Region', }, inplace = True) 

In [49]:
salary_drivers['NetAmount.1'].fillna(0, inplace=True)
salary_drivers['Multipl'].fillna(1, inplace=True)
salary_drivers['AdjAmount'] = salary_drivers['NetAmount.1'] + salary_drivers['NetAmount']

In [50]:
salary_drivers = salary_drivers.merge(TechnicalRouteDNN, left_on=['Region.Code'],right_on = ['Region.Code.1'], how='outer')

In [51]:
salary_drivers = salary_drivers.loc[salary_drivers['Month'] == month]

In [52]:
salary_drivers.rename(columns = {'Distance' : 'TechnicalRoute'}, inplace = True) 
salary_drivers['TechnicalRoute'].fillna(0, inplace=True)

In [53]:
salary_drivers = salary_drivers.drop(['AccType_y','Принадлежность ТС_y','Region_y','Milk_y','DNN/MC_y'], 1)

In [54]:
salary_drivers.rename(columns = {'Принадлежность ТС_x' : 'Принадлежность ТС', 'AccType_x' : 'AccType', 'Milk_x' : 'Milk', 'DNN/MC_x':'DNN/MC'}, inplace = True) 

In [55]:
def salary(salary_drivers):
    if (salary_drivers['Client'] == 'DNN T1'):
        return salary_drivers['Total km'] * salary_drivers['Multipl'] + salary_drivers['TechnicalRoute']
    else:
        return salary_drivers['Total km']* salary_drivers['Multipl']

In [56]:
salary_drivers['Adj total km'] = salary_drivers.apply(salary, axis=1)

In [57]:
ToDistNetAmount = (salary_drivers.groupby(['TechnicalRoute','Region.Code'])
                    .agg({'NetAmount.1':'sum', 'Adj total km':'sum'})
                    .reset_index())

In [58]:
salary_drivers = salary_drivers.merge(ToDistNetAmount, left_on=['Region.Code','TechnicalRoute'],right_on = ['Region.Code','TechnicalRoute'], how='outer')

In [59]:
salary_drivers.rename(columns = {'NetAmount.1_y' : 'ToDistNetAmount.1', 'Adj total km_y' : 'Region km'}, inplace = True) 

In [60]:
salary_drivers ['AdjsAmount'] = salary_drivers['NetAmount'] + salary_drivers['ToDistNetAmount.1']

In [61]:
salary_drivers ['ClientShare'] = salary_drivers['Adj total km_x'] / salary_drivers['Region km']

In [62]:
salary_drivers['PreAmount'] = salary_drivers['AdjsAmount'] * salary_drivers['ClientShare']

In [63]:
def salary_2(salary_drivers):
    if (salary_drivers['PreAmount'] == 0):
        return salary_drivers['NetAmount.1_x']
    else:
        return salary_drivers['PreAmount']

In [64]:
salary_drivers['Net Amount'] = salary_drivers.apply(salary_2, axis=1).abs()

In [65]:
#Расставим колонки в нужном порядке и укажем только те, что нам необходимы

salary_drivers = salary_drivers[['Month', 'AccType', 'Costs group', 'Номенклатурная группа', 'Calc items','NetAmount','Region.Code', 'Принадлежность ТС' ,'Client', 'DNN/MC','Total km','Sum of Total km','NetAmount.1_x', 'Multipl','AdjAmount','TechnicalRoute','Adj total km_x','ToDistNetAmount.1', 'Region km', 'AdjsAmount', 'ClientShare', 'PreAmount', 'Net Amount', 'Region', 'Milk']]

#### Сформируем данные по топливу fuel

In [66]:
fuel = upp_expenses[upp_expenses['Costs group'] == 'Fuel'] 
fuel = fuel.drop(['Client','Region','Code'], 1)

In [67]:
fuel = fuel.merge(uat_statistics, left_on=['Region.Code'],right_on = ['Region.Code'], how='outer')

In [68]:
fuel= fuel[fuel['Принадлежность ТС'] == 'Собственное']

In [69]:
fuel['% of Total km'] = fuel['Total km'] / fuel['Sum of Total km']

In [70]:
fuel['Net Amount'] = fuel['NetAmount'] * fuel['% of Total km']

In [71]:
fuel = fuel.drop(['Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 17'], 1)

Подготовка данных по Fuel завершена

#### Сформируем данные по лизингу

In [72]:
Leasing_special = manual[manual['Group'] == 'Leasing'] 

In [73]:
leasing_cost  = upp_expenses[upp_expenses['Costs group'] == 'Leasing'] 
leasing_cost = leasing_cost.drop(['Client','Region','Code'], 1)

In [74]:
leasing_cost = (leasing_cost.groupby(["Month", "Costs group", "Region.Code", "AccType"])
                    .agg({'NetAmount':'sum'})
                    .reset_index())

In [75]:
leasing_cost  = leasing_cost.merge(uat_statistics, left_on=['Region.Code'],right_on = ['Region.Code'], how='outer')

In [76]:
leasing_cost= leasing_cost[leasing_cost['Принадлежность ТС'] == 'Собственное']

In [77]:
leasing_cost  = leasing_cost.merge(Leasing_special, left_on=['Region.Code','Client', 'DNN/MC'],right_on = ['Region.Code','Client', 'DNN/MC'], how='outer')

In [78]:
leasing_cost = leasing_cost[['Month', 'Costs group', 'Region.Code','AccType_x','NetAmount_x','Принадлежность ТС_x' , 'Region_x', 'Milk_x','Client', 'DNN/MC','Turnover','Sum of Turnover', 'NetAmount_y','Multipl']]

In [79]:
leasing_cost['NetAmount_y'].fillna(0, inplace=True)
leasing_cost['Multipl'].fillna(1, inplace=True)

In [80]:
leasing_cost['Turnover_'] = leasing_cost['Turnover'] * leasing_cost['Multipl']

In [81]:
leasing_cost.rename(columns = {'NetAmount_x' : 'NetAmount', 'NetAmount_y_y' : 'Sum of NetAmount_','Turnover__y':'Sum of Turnover_'}, inplace = True) 

In [82]:
Inserted_Multiplication = (leasing_cost.groupby(['Region.Code'])
                    .agg({'NetAmount_y':'sum', 'Turnover_':'sum'})
                    .reset_index())

In [83]:
leasing_cost = leasing_cost.merge(Inserted_Multiplication, left_on=['Region.Code'],right_on = ['Region.Code'], how='outer')

In [84]:
leasing_cost.rename(columns = {'NetAmount_x' : 'NetAmount', 'NetAmount_y_y' : 'Sum of NetAmount_','Turnover__y':'Sum of Turnover_'}, inplace = True) 

In [85]:
leasing_cost['Adj_amount'] = leasing_cost['NetAmount'] + leasing_cost['Sum of NetAmount_']

In [86]:
leasing_cost['TurnoverShare'] = leasing_cost['Turnover__x'] / leasing_cost['Sum of Turnover_']

In [87]:
leasing_cost['PreAmount'] = leasing_cost['TurnoverShare'] * leasing_cost['Adj_amount']

In [88]:
def leasing(leasing_cost):
    if (leasing_cost['TurnoverShare'] == 0):
        return leasing_cost['NetAmount_y_x']
    elif (leasing_cost['Client'] == 'DNN T1'):
        return leasing_cost['PreAmount'] - leasing_cost['NetAmount_y_x']
    else:
        return leasing_cost['PreAmount']

In [89]:
leasing_cost['Net Amount'] = leasing_cost.apply(leasing, axis=1).abs()

In [90]:
leasing_cost.rename(columns = {'AccType_x' : 'AccType', 'Принадлежность ТС_x' : 'Принадлежность ТС','Region_x':'Region','Milk_x':'Milk','NetAmount_y_x':'NetAmount_','Turnover__x':'Turnover_'}, inplace = True) 

In [91]:
leasing_cost.head(2) #посмотрим на первые две строчки получаенного дата-фрейма (data-frame)

,Month,Costs group,Region.Code,AccType,NetAmount,Принадлежность ТС,Region,Milk,Client,DNN/MC,...,Sum of Turnover,NetAmount_,Multipl,Turnover_,Sum of NetAmount_,Sum of Turnover_,Adj_amount,TurnoverShare,PreAmount,Net Amount
0,12,Leasing,BE_M,Exp,"3,101,623.63",Собственное,BE,M,DNN Milk,DNN,...,353.15,0.00,1.00,353.15,0.00,353.15,"3,101,623.63",1.00,"3,101,623.63","3,101,623.63"
1,12,Leasing,BE_T,Exp,"20,144,965.05",Собственное,BE,T,Центр внедрения ПРОТЕК Фирма ЗАО,MC,...,"2,556.70",0.00,1.00,16.32,"-3,085,789.04","2,504.88","17,059,176.01",0.01,"111,145.35","111,145.35"


#### Расчет Fix Agency Cost

In [92]:
fix_agency_cost = upp_expenses[upp_expenses['Costs group'] == 'Fix agency cost'] 
fix_agency_cost['Принадлежность ТС'] = 'Собственное'

In [93]:
def fixagency(fix_agency_cost):
    if (fix_agency_cost['Номенклатурная группа'] == 'Данон'):
        return 'DNN'
    elif (fix_agency_cost['Номенклатурная группа'] == 'Центральный офис'):
        return 'DNN'
    else:
        return 'MC'

In [94]:
fix_agency_cost['DNN/MC'] = fix_agency_cost.apply(fixagency, axis=1)

In [95]:
fix_agency_cost.rename(columns = {'Code' : 'Milk', 'NetAmount' : 'Net Amount'}, inplace = True) 

In [96]:
fix_agency_cost.head(3)

,Month,AccType,Costs group,Номенклатурная группа,Client,Region,Milk,Calc items,Net Amount,Region.Code,Принадлежность ТС,DNN/MC
1,12,Exp,Fix agency cost,Данон,DNN Milk,BE,M,EBITDA,"1,583,284.86",BE_M,Собственное,DNN
3,12,Exp,Fix agency cost,Данон,DNN Milk,YA,M,EBITDA,"329,870.20",YA_M,Собственное,DNN
4,12,Exp,Fix agency cost,Данон,DNN T1,BE,T,DEPRECIATION,"61,659.66",BE_T,Собственное,DNN


In [97]:
four_employees = uat_statistics.copy(deep=False)

four_employees = four_employees[four_employees['Milk'] == 'T']
four_employees = four_employees[four_employees['Region'] != 'SP']

four_employees = (four_employees.groupby(["Region.Code", "Milk"])
                    .agg({'Total km':'sum'})
                    .reset_index())

In [98]:
total_distance = (four_employees.groupby(["Milk"])
                    .agg({'Total km':'sum'})
                    .reset_index())

In [99]:
total_distance

,Milk,Total km
0,T,"4,657,367.91"


In [100]:
four_employees = four_employees.merge(total_distance, on=['Milk'], how='outer')

In [101]:
four_employees.rename(columns = {'Total km_x' : 'Distance', 'Total km_y' : 'Total Distance'}, inplace = True) 

In [102]:
four_employees['Division'] = four_employees['Distance'] / four_employees['Total Distance']

In [103]:
four_empl_to_substract = 318434.128

In [104]:
four_employees['FourEmpAlloc'] = four_employees['Division'] * four_empl_to_substract

In [105]:
four_employees

,Region.Code,Milk,Distance,Total Distance,Division,FourEmpAlloc
0,BE_T,T,"3,165,322.21","4,657,367.91",0.68,"216,419.80"
1,SA_T,T,"131,058.80","4,657,367.91",0.03,"8,960.77"
2,YA_T,T,"1,360,986.90","4,657,367.91",0.29,"93,053.56"


### Total transportation

In [106]:
total_transportation =pd.concat([subcontractors_cost, repair_maintenance, salary_drivers, fuel, leasing_cost, fix_agency_cost, total_trs_revenue],axis=0)

In [107]:
total_transportation = total_transportation[["Costs group", "Принадлежность ТС", "Region", "Client", "Milk", "DNN/MC", "Region.Code",  "Net Amount"]]

In [108]:
total_transportation['Fin Amount'] = total_transportation['Net Amount'] * -1
total_transportation['Type'] = 'Exp'

In [109]:
total_transportation = (total_transportation.groupby(["Type", "Costs group", "Принадлежность ТС", "Region", "Client", "Milk", "DNN/MC", "Region.Code"])
                    .agg({'Net Amount':'sum', 'Fin Amount':'sum'})
                    .reset_index())

### Выгрузка полученных данных в рабочий файл excel

В самом начале мы вводили переменную writer, в которую положили путь: *S:\Actual\_Reporting 2022\22 04\Trs\Transport 22 PY.xlsx*

Теперь в этот рабочий файл внесем листы с полученными данными методом *to_excel*

In [110]:
uat_rev_exp.to_excel(writer, 'УПП_Expenses')
uat_statistics.to_excel(writer, 'UAT_Statistics')
manual.to_excel(writer, 'Manually')
total_transportation.to_excel(writer, 'Total Transportation')
bonus_variances.to_excel(writer, 'Bonus')
total_trs_revenue.to_excel(writer, 'TRS Revenue')
subcontractors_cost.to_excel(writer, 'Subco')
salary_drivers.to_excel(writer, 'SalaryDrivers')
fuel.to_excel(writer, 'Fuel')
repair_maintenance.to_excel(writer, 'Repair&Maintence')
leasing_cost.to_excel(writer, 'Leasing')
upp_expenses.to_excel(writer, 'УПП_Expenses')
four_employees.to_excel(writer, 'FourEmpl')

writer.save() #обязательно сохраним результат